In [1]:
import os 
os.chdir("/home/psenin/git/saxpy")
os.getcwd()

'/home/psenin/git/saxpy'

In [3]:
import numpy as np
from saxpy.sax import sax_via_window
from saxpy.saxvsm import series_to_wordbag
from saxpy.saxvsm import manyseries_to_wordbag

In [4]:
dat = np.array([0., 0., 0., 0., 0., -0.270340178359072, -0.367828308500142,
                0.666980581124872, 1.87088147328446, 2.14548907684624,
                -0.480859313143032, -0.72911654245842, -0.490308602315934,
                -0.66152028906509, -0.221049033806403, 0.367003418871239,
                0.631073992586373, 0.0487728723414486, 0.762655178750436,
                0.78574757843331, 0.338239686422963, 0.784206454089066,
                -2.14265084073625, 2.11325193044223, 0.186018356196443,
                0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.519132472499234,
                -2.604783141655, -0.244519550114012, -1.6570790528784,
                3.34184602886343, 2.10361226260999, 1.9796808733979,
                -0.822247322003058, 1.06850578033292, -0.678811824405992,
                0.804225748913681, 0.57363964388698, 0.437113583759113,
                0.437208643628268, 0.989892093383503, 1.76545983424176,
                0.119483882364649, -0.222311941138971, -0.74669456611669,
                -0.0663660879732063, 0., 0., 0., 0., 0.,])
sax1 = sax_via_window(dat, 6, 3, 3, "none", 0.01)
sax1

defaultdict(list,
            {'aac': [4, 10, 11, 30, 35],
             'abc': [12, 14, 36, 44],
             'acb': [5, 16, 21, 37, 43],
             'acc': [13, 52, 53],
             'bac': [3, 19, 34, 45, 51],
             'bba': [31],
             'bbb': [15, 18, 20, 22, 25, 26, 27, 28, 29, 41, 42, 46],
             'bbc': [2],
             'bca': [6, 17, 32, 38, 47, 48],
             'caa': [8, 23, 24, 40],
             'cab': [9, 50],
             'cba': [7, 39, 49],
             'cbb': [33],
             'cca': [0, 1]})

In [5]:
def series_to_wordbag(series, win_size, paa_size, alphabet_size=3,
                      nr_strategy='exact', z_threshold=0.01):
    """VSM implementation."""
    sax = sax_via_window(series, win_size, paa_size, alphabet_size,
                         nr_strategy, z_threshold)

    # convert the dict to a wordbag
    frequencies = {}
    for k, v in sax.items():
        frequencies[k] = len(v)
    return frequencies

In [6]:
def manyseries_to_wordbag(series_npmatrix, win_size, paa_size, alphabet_size=3,
                          nr_strategy='exact', z_threshold=0.01):
    """VSM implementation."""
    frequencies = {}

    for row in series_npmatrix:
        tmp_freq = series_to_wordbag(np.squeeze(np.asarray(row)), 
                                     win_size, paa_size, alphabet_size,
                                     nr_strategy, z_threshold)
        print tmp_freq
        for k, v in tmp_freq.items():
            if k in frequencies:
                frequencies[k] += v
            else:
                frequencies[k] = v

    return frequencies

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-6-ef8036735d4c>, line 10)

In [7]:
f1 = series_to_wordbag(dat, 6, 3, 3, "none", 0.01)
f1

{'aac': 5,
 'abc': 4,
 'acb': 5,
 'acc': 3,
 'bac': 5,
 'bba': 1,
 'bbb': 12,
 'bbc': 1,
 'bca': 6,
 'caa': 4,
 'cab': 2,
 'cba': 3,
 'cbb': 1,
 'cca': 2}

In [8]:
m = np.matrix([dat,dat])
m.shape
f2 = manyseries_to_wordbag(m, 6, 3, 3, "none", 0.01)
f2

{'aac': 10,
 'abc': 8,
 'acb': 10,
 'acc': 6,
 'bac': 10,
 'bba': 2,
 'bbb': 24,
 'bbc': 2,
 'bca': 12,
 'caa': 8,
 'cab': 4,
 'cba': 6,
 'cbb': 2,
 'cca': 4}

In [116]:
bag1 = { "this": 10, "is": 1, "a": 2, "sample": 1}
bag2 = { "this": 10, "is": 1, "another": 2, "example": 3}
bags = {"bag1":bag1, "bag2":bag2}
str(bags)

"{'bag2': {'example': 3, 'another': 2, 'is': 1, 'this': 10}, 'bag1': {'sample': 1, 'a': 2, 'is': 1, 'this': 10}}"

In [113]:
def bags_to_tfidf(bags_dict):
    """VSM implementation."""
    
    # classes
    count_size = len(bags_dict)
    classes = [*bags_dict.copy()]
    
    # word occurrence frequency counts
    counts = {}
    
    # compute frequencies
    idx = 0
    for name in classes:
        for word, count in bags_dict[name].items():
            if word in counts:
                counts[word][idx] = count
            else:
                counts[word] = [0] * count_size
                counts[word][idx] = count
        idx = idx + 1
    
    # compute tf*idf
    tfidf = {} # the resulting vectors dictionary
    idx = 0
    for word, freqs in counts.items():
        
        # document frequency
        df_counter = 0 
        for i in freqs:
            if i > 0:
                df_counter = df_counter + 1
        
        # if the word is everywhere, dismiss it
        if df_counter == len(freqs):
            continue
        
        # tf*idf vector
        tf_idf = [0.0] * len(freqs)
        i_idx = 0
        for i in freqs:
            if i != 0:
                tf = np.log(1 + i)
                idf = np.log(len(freqs) / df_counter)
                tf_idf[i_idx] = tf * idf
            i_idx = i_idx + 1
        
        tfidf[word] = tf_idf
            
        idx = idx + 1
        
    return {"vectors":tfidf, "classes":classes}
        

In [115]:
[*bags.copy()]

['2', '3', '1']

In [114]:
res = bags_to_tfidf(bags)
res

{'classes': ['2', '3', '1'], 'vectors': {}}

In [79]:
res["vectors"]

{'a': [0.761500010418809, 0.0],
 'another': [0.0, 0.761500010418809],
 'example': [0.0, 0.9609060278364028],
 'sample': [0.4804530139182014, 0.0]}

In [100]:
import re

def read_data_file(fname):
    
    data = []
    
    with open(fname, 'r') as fp:
        read_lines = fp.readlines()
        for line in read_lines:
            tokens = re.split("\\s+", line.strip())
            data_line = []
            for token in tokens:
                data_line.append(float(token))
            class_label = data_line.pop(0)
            data.append((class_label, data_line))
        
    res = {}
    for key, arr in data:
        key = str(int(key))
        if key in res.keys():
            res[key].append(arr)
        else:
            dat = [arr]
            res[key] = dat
    
    return res
            
dd = read_data_file('resources/data/cbf/CBF_TRAIN')

dd


{'1': [[-0.46427649,
   -0.55504787,
   -0.8428431,
   -0.86589548,
   -0.93639631,
   -0.81726995,
   -0.26361216,
   -1.2580483,
   -1.2503934,
   -0.91830825,
   -0.92210226,
   -0.98448828,
   -1.2880511,
   -1.14346,
   -1.0488611,
   -0.36538582,
   -0.6913952,
   -0.98055025,
   -0.99133601,
   -0.88708673,
   -1.109881,
   -0.7687248,
   -1.0102962,
   -0.84699606,
   -1.3277464,
   -0.7129093,
   -1.2977275,
   1.3314643,
   0.9357379,
   1.2360468,
   1.0560677,
   0.91074495,
   1.3713479,
   0.93705876,
   0.91029198,
   0.93108664,
   1.4937796,
   1.3192656,
   1.2190887,
   0.94836262,
   0.16902251,
   0.97739879,
   1.2301476,
   0.57156683,
   0.75134091,
   0.96185121,
   0.7813122,
   1.1044295,
   1.0003177,
   0.5507251,
   0.2688748,
   1.1255209,
   1.0758635,
   1.071738,
   1.1544819,
   0.75940186,
   0.1511864,
   1.0202768,
   1.2572739,
   1.007149,
   1.1624171,
   0.82999734,
   1.4850935,
   1.1050903,
   1.5393173,
   1.1156157,
   1.8913625,
   1.3900

In [101]:
for key, arr in dd.items():
    print(key)

2
3
1


In [118]:
bag1 = manyseries_to_wordbag(dd['1'], 6, 3, 3, "none", 0.01)
bag2 = manyseries_to_wordbag(dd['2'], 6, 3, 3, "none", 0.01)
bag3 = manyseries_to_wordbag(dd['3'], 6, 3, 3, "none", 0.01)
bag1

{'aac': 36,
 'abb': 27,
 'abc': 141,
 'aca': 25,
 'acb': 120,
 'acc': 38,
 'bab': 29,
 'bac': 123,
 'bba': 30,
 'bbb': 142,
 'bbc': 22,
 'bca': 107,
 'bcb': 21,
 'caa': 37,
 'cab': 117,
 'cac': 20,
 'cba': 127,
 'cbb': 27,
 'cca': 31}

In [119]:
bags = {'1' : bag1, '2' : bag2, '3' : bag3}
[*bags.copy()]

['2', '3', '1']

In [120]:
bags

{'1': {'aac': 36,
  'abb': 27,
  'abc': 141,
  'aca': 25,
  'acb': 120,
  'acc': 38,
  'bab': 29,
  'bac': 123,
  'bba': 30,
  'bbb': 142,
  'bbc': 22,
  'bca': 107,
  'bcb': 21,
  'caa': 37,
  'cab': 117,
  'cac': 20,
  'cba': 127,
  'cbb': 27,
  'cca': 31},
 '2': {'aac': 41,
  'abb': 30,
  'abc': 195,
  'aca': 27,
  'acb': 147,
  'acc': 43,
  'bab': 33,
  'bac': 139,
  'bba': 22,
  'bbb': 223,
  'bbc': 35,
  'bca': 131,
  'bcb': 30,
  'caa': 30,
  'cab': 139,
  'cac': 24,
  'cba': 113,
  'cbb': 27,
  'cca': 35},
 '3': {'aac': 19,
  'abb': 29,
  'abc': 87,
  'aca': 10,
  'acb': 71,
  'acc': 23,
  'bab': 20,
  'bac': 70,
  'bba': 27,
  'bbb': 152,
  'bbc': 20,
  'bca': 112,
  'bcb': 17,
  'caa': 28,
  'cab': 112,
  'cac': 17,
  'cba': 106,
  'cbb': 24,
  'cca': 32}}

In [121]:
vectors = bags_to_tfidf(bags)
vectors

{'classes': ['2', '3', '1'], 'vectors': {}}